In [1]:
import pandas as pd
import numpy as np

In [62]:
mutations = pd.read_csv('../Amy/mutations_filtered.csv', index_col = 0)
oncokb_genes = pd.read_csv('./cancerGeneList.tsv', sep = '\t')['Hugo Symbol'].unique()
patients = pd.read_csv('../Amy/patients.txt', header = None)

In [39]:
polyphen_mutations = mutations.loc[mutations['PolyPhen'].apply(lambda x: float(x.split('(')[-1].split(')')[0]) > 0.9 if x != '.' else False)]

In [94]:
grouped_by_gene = polyphen_mutations.groupby(by = 'Hugo_Symbol').agg({'patient_id' : 'nunique'}).reset_index()
top_genes = grouped_by_gene.loc[grouped_by_gene['patient_id'] > 10]
overlap_with_oncokb = list(set(top_genes['Hugo_Symbol'].unique()).intersection(set(oncokb_genes)))
genes_to_encode = top_genes.loc[top_genes['Hugo_Symbol'].apply(lambda x: x in overlap_with_oncokb)]

In [58]:
polyphen_mutations_to_encode = polyphen_mutations.loc[polyphen_mutations['Hugo_Symbol'].apply(lambda x: x in overlap_with_oncokb)]

In [74]:
by_patient_mutations = polyphen_mutations_to_encode.groupby(by = 'patient_id').agg({'Hugo_Symbol' : list}).reset_index()

In [91]:
gene_encoding = pd.get_dummies(by_patient_mutations['Hugo_Symbol'].explode())
gene_encoding['patient_id'] = by_patient_mutations['patient_id']

In [98]:
mutation_embedding = gene_encoding[['patient_id'] + overlap_with_oncokb]

In [100]:
mutation_embedding.to_csv('train_mutation_embedding.csv', index = False)